In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.applications.densenet import preprocess_input
import matplotlib.pyplot as plt 
import tensorflow
from keras.metrics import Recall,Precision

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
train_dataset = pd.read_csv('../input/digit-recognizer/train.csv')
test_dataset = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.shape

In [ ]:
from keras.utils import to_categorical

train_labels = np.asarray(train_dataset['label'])
train_labels = to_categorical(train_labels)

train_data = train_dataset.drop('label', axis=1)
train_data = np.asarray(train_data).reshape(train_data.shape[0], 28, 28, 1)

In [ ]:

from sklearn.model_selection import train_test_split

train_images, validation_images, train_labels, validation_labels = train_test_split(train_data, train_labels, test_size=0.3, random_state=40)

# data

In [ ]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=30, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=False, 
    fill_mode='constant')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow(train_images, train_labels, batch_size=256)
validation_generator = validation_datagen.flow(validation_images, validation_labels, batch_size=128)

# model

In [ ]:
model=Sequential()

# first CNN layer of a kernel size 100
model.add(Conv2D(100,(3,3),input_shape=(28,28,1)))
#Relu layer
model.add(Activation('relu'))
#MaxPooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# second convolution layer of a kernel size 50

model.add(Conv2D(50,(3,3)))
#Relu layer
model.add(Activation('relu'))
#MaxPooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatten layer 
model.add(Flatten())

# Dropout 
model.add(Dropout(0.3))

#FC layer of 64 neurons
model.add(Dense(64,activation='relu'))

#Softmax layer for two categories
model.add(Dense(10,activation='softmax'))


In [ ]:
# Summary

model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
Image('model.png',width=400, height=200)

In [ ]:
# Model Compile 

OPT    = tensorflow.keras.optimizers.Adam(lr=0.001)

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy',Precision(),Recall()],
              optimizer=OPT)

In [ ]:
# Defining Callbacks

filepath = './best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 25,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
model_history=model.fit(
                        train_generator, 
                        steps_per_epoch=100, 
                        epochs=50, 
                        validation_data=validation_generator, 
                        validation_steps=50, 
                        callbacks = callback_list,
                        verbose=1)

In [ ]:
import keras
model.save('./best_weights.hdf5')
#model = keras.models.load_model('../input/densenet/best_weights.hdf5')

# plot

In [ ]:
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('train set loss')

plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

plt.plot(model_history.history['precision'])
plt.plot(model_history.history['val_precision'])
plt.title(' precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()


plt.plot(model_history.history['recall'])
plt.plot(model_history.history['val_recall'])
plt.title(' recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()


# **evaluate**

In [ ]:
loss, acc , precision,recall=model.evaluate(validation_generator)
print('validation Accuracy: %.3f' % acc)
print('validation Precision: %.3f' % precision)
print('validation Recall: %.3f' % recall)
print('validation loss: %.3f' % loss)


# test

In [ ]:
test_images = np.asarray(test_dataset)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
test_images = test_images.astype('float32')/255

In [ ]:
predictions = model.predict(test_images, batch_size=128)
image_id = range(1, predictions.shape[0] + 1)

In [ ]:
pred = [np.argmax(i) for i in predictions]

In [ ]:
sample = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sample.head()

In [ ]:
submission = pd.DataFrame({'ImageId': image_id, 'Label': pred})
submission.to_csv('digit_recognizer_submission', index=False)
print('file saved!')

# output